## Getting Started

### Prerequisites
1. Create an Ollama account at [ollama.com](https://ollama.com)
2. Generate an API key from [ollama.com/settings/keys](https://ollama.com/settings/keys)
3. Set the API key in the `.env` file in this folder

### Installation
```bash
pip install ollama python-dotenv requests
```

In [5]:
# Setup: Install required packages and import libraries
import sys
import subprocess

packages = ['ollama', 'python-dotenv', 'requests']

for package in packages:
    try:
        __import__(package.replace('-', '_'))
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("✓ All packages installed successfully!")

Installing python-dotenv...
✓ All packages installed successfully!


In [6]:
# Import all required libraries
import os
from pathlib import Path
from dotenv import load_dotenv
import ollama
from typing import Dict, Any

# Load environment variables from .env file
# Use Path.cwd() since __file__ is not defined in Jupyter notebooks
env_path = Path.cwd() / '.env'
if env_path.exists():
    load_dotenv(env_path)
else:
    # Try to find .env in the ollama-cloud directory
    load_dotenv(dotenv_path='d:\\ollama-n8n\\ollama-cloud\\.env')

# Get API key and configuration
OLLAMA_API_KEY = os.getenv('OLLAMA_API_KEY')
OLLAMA_ENDPOINT = os.getenv('OLLAMA_API_ENDPOINT', 'https://ollama.com')
DEFAULT_MODEL = os.getenv('DEFAULT_MODEL', 'gpt-oss:120b')

# Verify API key is set
if not OLLAMA_API_KEY or OLLAMA_API_KEY == 'your_api_key_here':
    print("  WARNING: OLLAMA_API_KEY not set in .env file!")
    print("Please set your API key in the .env file: https://ollama.com/settings/keys")
else:
    print(f"✓ Ollama API configured")
    print(f"  Endpoint: {OLLAMA_ENDPOINT}")
    print(f"  Default Model: {DEFAULT_MODEL}")

# Initialize Ollama client for cloud API
client = ollama.Client(
    host=OLLAMA_ENDPOINT,
    headers={'Authorization': f'Bearer {OLLAMA_API_KEY}'}
)

✓ Ollama API configured
  Endpoint: https://ollama.com
  Default Model: gpt-oss:120b


## Advanced: Building an AI Agent with Web Search

Combine thinking, tool calling, and web search to create an intelligent agent that can research topics and provide comprehensive answers.

In [7]:
def test_ai_agent_with_search(model: str = 'gpt-oss:120b', query: str = "What are the latest developments in Ollama as of 2025?") -> Dict[str, Any]:
    """
    Advanced: Build an AI agent that uses web search and thinking to answer questions.
    
    Args:
        model: Model with thinking support and tool access
        query: Question for the agent to research
    
    Returns:
        Agent's researched response
    """
    
    print(f"   Building AI Research Agent")
    print(f"   Model: {model}")
    print(f"   Query: '{query}'\n")
    
    # Define available tools for the agent
    available_tools = {
        'web_search': ollama.web_search,
        'web_fetch': ollama.web_fetch
    }
    
    # Tool definitions for the model
    tools = [
        {
            "type": "function",
            "function": {
                "name": "web_search",
                "description": "Search the web for information",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {"type": "string", "description": "Search query"},
                        "max_results": {"type": "integer", "description": "Max results (default 3)"}
                    },
                    "required": ["query"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "web_fetch",
                "description": "Fetch content from a specific URL",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "url": {"type": "string", "description": "URL to fetch"}
                    },
                    "required": ["url"]
                }
            }
        }
    ]
    
    messages = [{'role': 'user', 'content': query}]
    
    # Think level for GPT-OSS, boolean for others
    think_param = 'medium' if 'gpt-oss' in model else True
    
    print(f"Agent reasoning process...\n")
    
    # Main agent loop
    iteration = 0
    max_iterations = 3  # Prevent infinite loops
    
    while iteration < max_iterations:
        iteration += 1
        print(f"[Iteration {iteration}]")
        
        # Get response with tools enabled
        response = client.chat(
            model=model,
            messages=messages,
            tools=tools,
            think=think_param,
            stream=False
        )
        
        # Show thinking if available
        if hasattr(response.message, 'thinking') and response.message.thinking:
            thinking = response.message.thinking[:200] + "..." if len(response.message.thinking) > 200 else response.message.thinking
            print(f"  Thinking: {thinking}")
        
        # Show content
        if response.message.content:
            print(f"  Response: {response.message.content[:200]}...")
        
        # Check for tool calls
        if hasattr(response.message, 'tool_calls') and response.message.tool_calls:
            print(f"  Tools to call: {len(response.message.tool_calls)}")
            
            # Append assistant's response
            messages.append(response.message)
            
            # Execute tools
            for tool_call in response.message.tool_calls:
                func_name = tool_call.function.name
                func_args = tool_call.function.arguments
                
                print(f"    → Calling {func_name}...")
                
                try:
                    # Call the function
                    if func_name == 'web_search':
                        result = client.web_search(
                            func_args.get('query', ''),
                            max_results=func_args.get('max_results', 3)
                        )
                        result_str = str(result)[:1500]  # Limit context
                    elif func_name == 'web_fetch':
                        result = client.web_fetch(func_args.get('url', ''))
                        result_str = str(result)[:1500]  # Limit context
                    else:
                        result_str = "Tool not found"
                    
                    # Add tool result to conversation
                    messages.append({
                        'role': 'tool',
                        'content': result_str,
                        'tool_name': func_name
                    })
                    print(f"      Result: {result_str[:100]}...")
                
                except Exception as e:
                    print(f"      Error: {str(e)}")
                    messages.append({
                        'role': 'tool',
                        'content': f"Error: {str(e)}",
                        'tool_name': func_name
                    })
        
        else:
            # No more tool calls, agent has completed
            print("\n✓ Agent completed research\n")
            break
        
        print()
    
    # Return final response
    final_content = response.message.content if response.message.content else "Agent completed"
    print(f"Final Answer:\n{final_content}\n")
    
    return {
        'query': query,
        'iterations': iteration,
        'final_response': final_content
    }

# Run the AI agent
print("=" * 70)
print("AI RESEARCH AGENT - Web Search & Thinking Combination")
print("=" * 70 + "\n")

agent_result = test_ai_agent_with_search(
    query="What is Ollama and what are its key advantages?"
)

AI RESEARCH AGENT - Web Search & Thinking Combination

   Building AI Research Agent
   Model: gpt-oss:120b
   Query: 'What is Ollama and what are its key advantages?'

Agent reasoning process...

[Iteration 1]
  Thinking: The user asks: "What is Ollama and what are its key advantages?" Need to provide a comprehensive answer. Probably about Ollama, an open-source platform for running LLMs locally. Provide description, f...
  Response: **Ollama – at a glance**

Ollama is an open‑source platform that makes it painless to run Large Language Models (LLMs) on your own hardware (desktop, laptop, on‑prem server, or cloud VM). It bundles a...

✓ Agent completed research

Final Answer:
**Ollama – at a glance**

Ollama is an open‑source platform that makes it painless to run Large Language Models (LLMs) on your own hardware (desktop, laptop, on‑prem server, or cloud VM). It bundles a lightweight model‑serving daemon, a simple command‑line client, and a set of utilities for downloading, managing,